In [1]:
# Import Distributions to generate random numbers W matrix of the RBM
using Distributions
using MNIST
using BenchmarkTools
using PyPlot
using Combinatorics
#using ImageView

In [2]:
X_train, y_train = MNIST.traindata()
X_test, y_test = MNIST.testdata()

T = Float32
X_train = Array{T}( (X_train - minimum(X_train))/(maximum(X_train) - minimum(X_train)) )
y_train = Array{T}(y_train)
X_test = Array{T}(X_test - minimum(X_test))/(maximum(X_test) - minimum(X_test)) 
y_test = Array{T}(y_test);

In [15]:

function sigmoid(x::Float32)
    return 1/(1 + exp(-x))
end

type RBM{T <: Real}
    n_vis::Int
    n_hid::Int
    W::Matrix{T}
    vis_bias::Vector{T}
    hid_bias::Vector{T}
    trained::Bool
    n_epochs_trained::Int
end

function initialize_RBM(n_vis, n_hid, sigma, T)
    
    return RBM{T}(n_vis,                                 # num visible units 
                  n_hid,                                 # num hidden unnits
                  rand(Normal(0,sigma), n_hid, n_vis),  # weight matrix
                  zeros(n_vis),                          # visible vector  
                  zeros(n_hid),                          # Hidden vector
                  false,0)                                 # trained
end

function Base.show{T}(io::IO, rbm::RBM{T})
    n_vis = size(rbm.vis_bias, 1)
    n_hid = size(rbm.hid_bias, 1)
    trained = rbm.trained
    print(io, "RBM{$T}(n_vis=$n_vis, n_hid=$n_hid, trained=$trained)")
end

function generate_M(W, n_columns)

    n_hid = size(W)[1]
    n_vis = size(W)[2]
    costat = Int(sqrt(n_vis))    
    n_rows = Int(round(n_hid/n_columns))   

    print("\ncostat: ",costat,
          " \nn_rows: ", n_rows,
          "\nn_cols: ", n_columns)
    
    M = zeros(costat * n_rows, costat * n_columns)
    
    n_im = 1
    for r in 1:costat:size(M)[1]
        for c in 1:costat:size(M)[2]
            M[r:r+costat-1, c:c+costat-1] = reshape(W[n_im,:],costat,costat)
            n_im +=1
        end
    end
    return M
end


function fit!(rbm, X, batch_size::Integer,  n_epochs::Integer, K::Integer, lr::Real,T, shuffle_data)
        
    n_samples = size(X)[2]
    indicies = [x:min(x + batch_size-1, n_samples) for x in 1:batch_size:n_samples]
    sample_perm = Vector(1:n_samples)
    n_minibatches = Float32(length(indicies))
    rec_errors = Vector{Float32}([])
   
    for epoch in 1:n_epochs
        rec_error = Float32(0.)
        
        if shuffle_data==true
            shuffle!(sample_perm)
            X .= X[:,sample_perm]
        end
        
        for minibatch_ind in indicies          
            rec_error += partial_fit!(rbm, X[:, minibatch_ind], K, lr,T)
        end
        
        push!(rec_errors, rec_error/n_minibatches)
        print(rec_errors[end], "\n")
        rbm.n_epochs_trained +=1
    end
    rbm.trained = true
    return rec_errors
end


function partial_fit!(rbm, X, K::Integer, lr::Real, T)
    batch_size = size(X)[2]   
    local H_hat::Array{T} = zeros(rbm.n_hid, batch_size)
    local V_hat::Array{T} = zeros(rbm.n_vis, batch_size)

    for k in 1:K
        if k ==1
            H = sigmoid.(rbm.W * X .+ rbm.hid_bias)
            V_hat .= sigmoid.(rbm.W' * H .+ rbm.vis_bias) .> rand(rbm.n_vis, batch_size)
            H_hat .= sigmoid.(rbm.W * V_hat .+ rbm.hid_bias)
        else
            V_hat .= sigmoid.(rbm.W' * H_hat .+ rbm.vis_bias) .> rand(rbm.n_vis, batch_size)
            H_hat .= sigmoid.(rbm.W * V_hat .+ rbm.hid_bias)
        end
    end
   
    rbm.W .+= lr.*(H * X' .- H_hat*V_hat') ./ batch_size; 
    rbm.vis_bias .+= lr .* vec(sum((X .- V_hat), 2))./ batch_size;
    rbm.hid_bias .+= lr .* vec(sum((H .- H_hat), 2)) ./ batch_size;
    return sqrt(sum((X.-V_hat).^2))
end


partial_fit! (generic function with 1 method)

In [16]:
rbm = initialize_RBM(784, 20, 0.01, Float32)

RBM{Float32}(n_vis=784, n_hid=20, trained=false)

In [17]:
eltype(rbm.vis_bias)

Float32

In [18]:
eltype(rbm.hid_bias)

Float32

In [64]:
@time partial_fit!(rbm, X_train[:,1:500], 1, 0.1, Float32)

  0.027971 seconds (515 allocations: 18.733 MB, 2.89% gc time)


196.4948f0

In [65]:
@benchmark partial_fit!(rbm, X_train[:,1:500], 1, 0.1, Float32)

BenchmarkTools.Trial: 
  memory estimate:  18.73 MiB
  allocs estimate:  511
  --------------
  minimum time:     20.372 ms (0.00% GC)
  median time:      21.550 ms (2.82% GC)
  mean time:        26.188 ms (1.95% GC)
  maximum time:     85.973 ms (0.73% GC)
  --------------
  samples:          191
  evals/sample:     1
  time tolerance:   5.00%
  memory tolerance: 1.00%

In [47]:
n_epochs = 1
batch_size = 500
K = 1
lr = 0.05

@benchmark fit!(rbm, X_train, batch_size,  n_epochs, K, lr, T, false)

204.22185
203.31963
202.5565
201.79709
201.1657


BenchmarkTools.Trial: 
  memory estimate:  2.20 GiB
  allocs estimate:  61808
  --------------
  minimum time:     3.621 s (1.85% GC)
  median time:      3.916 s (1.73% GC)
  mean time:        3.916 s (1.73% GC)
  maximum time:     4.210 s (1.62% GC)
  --------------
  samples:          2
  evals/sample:     1
  time tolerance:   5.00%
  memory tolerance: 1.00%

In [ ]:
n_epochs = 10
batch_size = 500
K = 1
lr = 0.01

res = fit!(rbm, X_train, batch_size,  n_epochs, K, lr, T, false)

In [ ]:
n_epochs = 1
batch_size = 500
K = 2
lr = 0.001

@benchmark fit!(rbm, X_train, batch_size,  n_epochs, K, lr, T, false)

In [ ]:
n_epochs = 1
batch_size = 500
K = 3
lr = 0.001

@benchmark fit!(rbm, X_train, batch_size,  n_epochs, K, lr, T, false)

In [ ]:
n_epochs = 1
batch_size = 500
K = 10
lr = 0.001

@benchmark fit!(rbm, X_train, batch_size,  n_epochs, K, lr, T, false)

In [ ]:
n_epochs = 10
batch_size = 500
K = 2
lr = 0.001

res = fit!(rbm, X_train, batch_size,  n_epochs, K, lr, T, true)

In [ ]:
PyPlot.plot(res)

In [ ]:
rbm = initialize_RBM(784, 20, 0.01, Float32)

n_epochs = 40
batch_size = 500
K = 3
lr = 0.001

res = fit!(rbm, X_train, batch_size,  n_epochs, K, lr, T, true)

In [ ]:
PyPlot.plot(res)

In [ ]:
for i in 1:1
    print("HI")
end

In [ ]:
M = generate_M(rbm.W,25);
PyPlot.imshow(M,"gray")

In [ ]:
M = generate_M(rbm.W, 10);

PyPlot.imshow(M,"gray")

In [ ]:
Hneg = zeros(rbm.n_hid, 10);

In [ ]:
Hneg .= sigmoid.(rbm.W * X_train[:,1:10] .+ rbm.hid_bias) .> rand(rbm.n_hid, 10);

In [ ]:
sum(Hneg)

In [ ]:
rand()

### Function to plot the weights

In [ ]:
# https://github.com/dfdx/Boltzmann.jl/blob/master/examples/mnistexample.jl

function plot_weights(W, imsize)
    padding=10
    h, w = imsize
    n = size(W, 1)
    rows = Int(floor(sqrt(n)))
    cols = Int(ceil(n / rows))
    halfpad = div(padding, 2)
    dat = zeros(rows * (h + padding), cols * (w + padding))
    for i=1:n
        wt = W[i, :]
        wim = reshape(wt, imsize)
        wim = wim ./ (maximum(wim) - minimum(wim))
        r = div(i - 1, cols) + 1
        c = rem(i - 1, cols) + 1
        dat[(r-1)*(h+padding)+halfpad+1 : r*(h+padding)-halfpad,
            (c-1)*(w+padding)+halfpad+1 : c*(w+padding)-halfpad] = wim
    end
    #ImageView.view(dat)
    return dat
end

In [ ]:
rbm = initialize_RBM(784, 50, 0.01, Float32)
M = generate_M(rbm.W,25);
PyPlot.imshow(M,"gray")

## Train model several epochs 

Plot the weights after training

In [ ]:
rbm = initialize_RBM(784, 50, 0.01, Float32)
M = generate_M(rbm.W,25);
PyPlot.imshow(M,"gray")

In [ ]:
n_epochs = 100
batch_size = 200
K = 1
lr = 0.01

fit!(rbm, X_train,  batch_size,  n_epochs, K, lr, T, true)

In [ ]:
M = generate_M(rbm.W, 10);
PyPlot.imshow(M,"gray")

In [ ]:
PyPlot.imshow(M,"gray", vmin=minimum(M), vmax = maximum(M))

In [ ]:
dat = plot_weights(rbm.W[1:50,:], (28,28));
PyPlot.imshow(dat,"gray")

## Scaling individually the features of the plot: TO DO

#### Python code for printing the feature detectors


    for i, comp in enumerate(self.W.T):
        plt.subplot(15, 15, i + 1)
        if min_max_scale:
            plt.imshow(comp.reshape((28, 28)),
                       cmap= plt.get_cmap('gray'), vmin=min_, vmax=max_)

In [ ]:
function plot_per_hidden_unit(W, n_columns, minmax_scale)
    n_hid = size(W)[1]
    n_vis = size(W)[2]
    costat = Int(sqrt(n_vis))    
    n_rows = Int(round(n_hid/n_columns))   

    print("\ncostat: ", costat,
          "\nn_rows: ", n_rows,
          "\nn_cols: ", n_columns)
    
    if minmax_scale == true
        min_ = minimum(W)
        max_ = maximum(W)
    end
                    
    for i in 1:n_hid
        comp = W[i,:]
        PyPlot.subplot(10, 10, i+1)
        if minmax_scale == true
            PyPlot.imshow(reshape(comp, 28, 28),
            cmap= PyPlot.get_cmap("gray"), vmin=min_, vmax=max_)
        end
    end
    
end

In [ ]:
plot_per_hidden_unit(rbm.W, rbm.n_hid, true)